In [1]:
import sys
import copy
import random
import hashlib
import cupy as cp 
import numpy as np
from math import pi
import numpy.linalg as LA
from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('../..')))

from datetime import datetime
from os.path import expanduser

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from LevelSetPy.Grids import *
from LevelSetPy.Utilities import *
from LevelSetPy.Visualization import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration import *
from BRATVisualization.rcbrt_visu import RCBRTVisualizer

%matplotlib inline

In [2]:
args = Bundle(dict(visualize=True, flock_num=4, resume="murmurations_flock_01_02-06-22_17-43.hdf5", flock_payoff=False, pause_time=1e-5))

Dynamical System

\begin{align}
  \dot{x}_1 = &-v_e + v_p cos x_3 + w_e x_2  \\
                &\dot{x}_2 = -v_p sin x_3 - w_e x_1  \\
                &\dot{x}_3 = -w_p - w_e
\end{align}


\begin{align}
  \dot{x}_1 = f(x) + f(x, u) + f(x,v)
\end{align}

### Hamiltonian

$$ H(x,p) = p_1 [v_e - v_p cos(x_3)] - p_2 [v_p sin x_3] 
                   - w | p_1 x_2 - p_2 x_1 - p_3| + w |p_3|$$

$$u = \begin{bmatrix}  
        0 \\ 0 \\ -w_e
      \end{bmatrix}, v = \begin{bmatrix}  
        0 \\ 0 \\ -w_p
      \end{bmatrix} $$                

Find $H_u$, $H_v, \, H_{uu}, H_{vv}, H_{uv}, H_{vu}, H_x, H_{xx} $    

Observe: $v_e = v_p = 1$ <--- Not input

$w_e = w_p = +1$ <-- input


$$ H(x,p) = p_1 [1 -  cos(x_3)] - p_2 [ sin x_3] 
                   - w_e | p_1 x_2 - p_2 x_1 - p_3| + w_p |p_3|$$

Rewrite in terms of $u $ and $v$:

$$ H(x,p) = p_1 [1 -  cos(x_3)] - p_2 [ sin x_3] 
                   - u | p_1 x_2 - p_2 x_1 - p_3| + v |p_3|$$
                 


$$H_u = -| p_1 x_2 - p_2 x_1 - p_3|, \,\,H_v = |p_3|$$ 


$$
        H_{ux} = \begin{bmatrix}
                        -sgn(p_2 x_1) \mid p_2 \mid \\
                        -sgn(p_1 x_2) \mid p_1 \mid \\
                        0 
                \end{bmatrix} \,\,
        H_{vx} = \begin{bmatrix}
                        0 \\
                        0 \\
                        0 
                \end{bmatrix}
$$

$$H_{uu} = 0, \,\, H_{vv} = 0$$

$$H_{x} = \begin{bmatrix}
                -sgn(p_2 x_1) \mid p_2 \mid u \\
                sgn(p_1 x_2) \mid p_1 \mid u \\
                p_1 \sin x_3 - p_2 \cos x_3
        \end{bmatrix}
        $$    

$$H_{xx} = \begin{bmatrix}
                -2 \mid p_2  \mid \delta( x_1) u & 0 & 0 \\
                0 & 2 \mid p_1 \mid \delta(x_2)  u & 0\\
                0 & 0 & p_1 \cos x_3 + p_2 \sin x_3
        \end{bmatrix}
        $$    

In [3]:
grid_min = expand(np.array((-.75, -1.25, -np.pi)), ax = 1)
grid_max = expand(np.array((3.25, 1.25, np.pi)), ax = 1)
pdDims = 2                      # 3rd dimension is periodic
resolution = 100
N = np.array(([[
                resolution,
                np.ceil(resolution*(grid_max[1, 0] - grid_min[1, 0])/ \
                            (grid_max[0, 0] - grid_min[0, 0])),
                resolution-1
                ]])).T.astype(int)
grid_max[2, 0] *= (1-2/N[2,0])
g = createGrid(grid_min, grid_max, N, pdDims)

In [4]:
"""
    What we do here is really simple. We run as many 
    a fwd and backward pass of ddp algos as possible  
    on trajectories that are finely spaced apart by g.dx 
    
    This part accumulates all trajectories within grid bounds
"""
V_buf = []
all_traj = [copy.deepcopy(grid_min)]
while np.any(all_traj[-1]<grid_max):
    all_traj += [all_traj[-1]+g.dx]

all_traj = np.array(all_traj[:resolution]).squeeze()

# fix the inconsistencies in (x1, x2, x3)
indices = [np.nan for idx in range(len(all_traj))]
for dim in range(all_traj.shape[-1]):
    indices[dim] = all_traj[:,dim]>grid_max[dim,0]
    
# replace trajectories with bounds that exceed max 
# values along respective dimensions
for dim in range(all_traj.shape[-1]):
    all_traj[indices[dim],dim] = grid_max[dim,0]

In [ ]:
x = [1.5, 2.5, 3.0] 
dx = .1

grid_min = [1, 2, 2.5]
grid_max = [2, 3.5, 4.5]

X = [grid_min]
for i < N:
    X += X[-1]+dx

In [5]:
"""
    Once we have the trajectories, we can start with the 
    usual backward and forward pass of DDP.
    
    This will require two loops:
        Upper loop: Whereupon we iterate throuh every possible trajectory in 
        the system
        Lower loop: Forward and backward passes of DDP
"""
x = copy.copy(all_traj)
xr = 
# Here we go:
for traj_idx, traj in enumerate(all_traj):
    

[-0.75       -1.25       -3.14159265]
[-0.70959596 -1.20967742 -3.07812614]
[-0.66919192 -1.16935484 -3.01465962]
[-0.62878788 -1.12903226 -2.9511931 ]
[-0.58838384 -1.08870968 -2.88772658]
[-0.5479798  -1.0483871  -2.82426006]
[-0.50757576 -1.00806452 -2.76079354]


In [39]:
grid_max.T, grid_max.shape

(array([[3.25      , 1.25      , 3.07812614]]), (3, 1))

### Polytope test

Given a matrix $A \in \mathbb{R}^n$ and a vector $b \in \mathbb{R}^m$, an $\mathcal{H}-$ polyhedron is

\begin{align}
    \mathcal{P} = \left\{x \in \mathbb{R}^n \mid Ax \le b \right\}
\end{align}

In [33]:
# x = np.linspace(0.1, 10, 3)

A = np.array([
     [.3, .7, 5.5],
     [8.4, 2.3, 6.7],
     [1.9, 2.5, 8.8]])

b = [1, 2, 3]


# Compute various x's
import numpy.linalg as LA 
Xes = []




array([[3.0000e-02, 3.5350e+00, 5.5000e+01],
       [8.4000e-01, 1.1615e+01, 6.7000e+01],
       [1.9000e-01, 1.2625e+01, 8.8000e+01]])